# Prep abstracts for LDA

The overall goal for this script is to filter through each year's worth of abstracts from 1985-2016 that are **(1) neuroscience-related** and **(2) R01s**.

For each abstract that met those conditions, I decomposed that abstract into a list of words that were lowercase, with punctuation removed, stemmed, and also removed the stopwords. I then saved each wordlist, with the corresponding year, into a csv file, *all_years.csv*, containing abstract wordlists for 1985-2016.

In [1]:
# import some packages
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot

from string import punctuation
from nltk.corpus import stopwords
from nltk import PorterStemmer

%matplotlib inline

# Read abstracts and project csvs into dataframes

In [2]:
# initialize a dataframe
initialize_df = pd.DataFrame(columns=['wordlist', 'year'])

In [3]:
# write dataframe to csv file
try: 
    initialize_df.to_csv('all_years.csv')
except: 
    pass

In [4]:
def get_abstracts_projects(year):
    """
    input: int -- year to analyze; [1985:2016] inclusive.
    returns: 2 dataframes -- project & abstract
    """
    abs_dir = './abstracts/'
    proj_dir = './projects/'
    
    project = pd.read_csv(proj_dir + 'RePORTER_PRJ_C_FY' + year + '.csv', encoding = "ISO-8859-1")
    abstract = pd.read_csv(abs_dir + 'RePORTER_PRJABS_C_FY' + year + '.csv',encoding = "ISO-8859-1")
    
    return project, abstract

# Clean and filter for neuro abstracts

### Clean abstracts (stemming, lowercase, no punctuation, remove stopwords)

In [5]:
# get customized stopwords:
def customized_stopwords(to_append=[]):
    """
    to_append: list; what words do you want to exclude from your analysis, in addition to the standard 
    stopwords like 'the', 'and, 'of', and so on? See above to_append variable for examples.
    returns: list; stopwords including to_append list
    """
    stop = stopwords.words('english')
    stop = stop + to_append
    return stop

# get list of words that are lowercase, with punctuation removed and words stemmed.
def get_wordlist(abstract, stop = customized_stopwords()):
    """
    returns a list of lowercase words from abstract with punctuation and stopwords removed.
    """
    try:
        # make words lowercase
        words = abstract.lower()
        
        # take out all punctuation and split strings into a list of words
        words = (''.join(c for c in words if c not in punctuation)).split(' ')
        
        # remove stopwords
        words = [" ".join([w for w in word.split() if not w in stop]) for word in words]
        
        # stem words using Porter's Stemmer
        stemmed = []
        for word in words:
            try:
                word = PorterStemmer().stem(word)
            except IndexError:
                word = word
            if word != '' and word.isalpha():
                stemmed.append(word)
        words = stemmed

    except AttributeError:
        words = []
    return words

### Filter for neuro-related abstracts.
I defined a project to be neuroscience-related if the abstract mentioned "brain" or "neur*" at least once every 100 non-stopwords.

In [6]:
def neuro_count(row):
    try:
        return (row.ABSTRACT_TEXT.count(' brain') + row.ABSTRACT_TEXT.count('neur'))
    except AttributeError:
        return 0
    
def wordlist_count(row):
    return len(row.wordlist)
    
def neuro_only(df, word_density=0.01):
    """
    input: dataframe
    word_density: how many neuro related words for every 100 words that are not stopwords in an abstract? Stopwords: the, and, or, not, etc.
    returns: dataframe containing neuro-related projects as defined above, with a column containing cleaned abstract keywords for analysis.
    """
    df['abs_neuro_count'] = df.apply(neuro_count, axis=1)
    df['wordlist'] = df.ABSTRACT_TEXT.apply(get_wordlist)
    df['wordlist_ct'] = df.apply(wordlist_count, axis=1)
    df['rel_neuro_count'] = df.abs_neuro_count / df.wordlist_ct
    
    # drop duplicates - I found that in some years, there are duplicate abstracts which really screwed up my hierarchical clustering
    df = df.drop_duplicates('ABSTRACT_TEXT')
    return df[df.rel_neuro_count >= word_density]

# Implementation

In [9]:
years = range(1985,2017)
grant_type = 'R01'

# get abstracts and projects for a single year
for year in years:
    project, abstract = get_abstracts_projects(str(year))

    # join abstracts to projects dataframe
    df = project.merge(abstract, on='APPLICATION_ID', how='left')
    
    # Implement clean abstracts and filter for neuro projects.
    df_neuro = neuro_only(df)

    # look only at R01s
    df_neuro_granttype = df_neuro[df_neuro.ACTIVITY == grant_type]

    # save wordlist and year to csv file
    df_to_save = pd.DataFrame(df_neuro_granttype['wordlist'])
    df_to_save['year'] = year
    df_to_save.to_csv('all_years.csv', mode='a', header=False)
    
    print (year, 'done')
print ('all done')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


1996 done
all done
